## Capturing an image from a webcam

In your course, you might want to have students use their own webcam. As an example, if you are teaching object detection methods, it may be nice if students can experiment with this on frames that they captured themselves.

In principle, all of the python code in a Jupyter Notebook is executed on the so-called 'server side'. I.e. that means it runs on one of the servers from SURF. That is convenient for many things (for example it allows all your students to use exactly the same software environment), but it is a challenge if you want to use local resources such as a webcam.

The way in which this can be achieved is by executing Javascript code. Javascript code can be executed on the 'client side', in the browser. That is: it is executed by the browser on the laptop or deskop of the student/teacher. There is one downside to this: browsers may differ, and Javascript code might not work equally well in all browsers. The following code has been tested on Google Chrome, version 107.0.5304.107.

First, some imports

In [ ]:
from IPython.display import display, Javascript, Image
from base64 import b64decode

Since the javascript code (that we'll define later) will return a base64-encoded image, we specify a function that can save an image to disk based on a base64-encoded image as argument. Note that this function will save the file in the current working directory, on the 'server-side'. That is, the image will be stored on the Jupyter Notebook Server, not on the local laptop/desktop of the end user.

In [ ]:
def save_photo(filename, photo_b64):
    print("Saving photo: {filename}".format(filename=filename))
    binary = b64decode(photo_b64.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

Then, we define the function that can capture the image from the webcam. The function 'take_photo' is a python function, that essentially wraps Javascript code and executes that Javascript code. In the javascript code, we call the save_photo() function we defined above, so that the photo is saved after it is captured.

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
    takePhotoJS='''async function takePhoto(filename, quality){
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);
 
      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      const imageURL = canvas.toDataURL('image/jpeg', quality)
      
      // Save photo by calling the 'save_photo' python function we defined earlier
      var command = "save_photo('" + filename+ "', '" + imageURL + "')"
      var kernel = IPython.notebook.kernel;
      kernel.execute(command);
      
      return imageURL;
    }'''
    callTakePhoto='\nconst imageURL = takePhoto("{filename}", {quality})'.format(filename=filename, quality = quality)

    # This call defines the javascript script we'd like to execute
    js = Javascript(takePhotoJS + callTakePhoto)
    
    # This call executes the javascript script
    display(js)
    
    return filename

Then, we define a small function that can display the photo:

In [ ]:
def show_photo(filename='photo.jpg'):
    display(Image(filename))

Now, it's time to call the functions we just created. First, we call the function above to take an actual photo with the webcam. Note that this opens a screen (at the bottom of this notebook, so make sure to scroll down all the way!) displaying the webcam video, along with a 'capture' button. Press the capture button to take a photo.

In [ ]:
filename = take_photo()

Next, we call the show_photo function, to display the photo

In [ ]:
show_photo(filename)

Note that we should not combine the 'take_photo()' and 'show_photo()' functions in the same codeblock: the take_photo function is a so-called 'asynchronous' function. It will return immediately, even if it has not completed yet. That means that if we were to combine the two functions in the same codeblock, the 'show_photo()' function would be called before you would have had a chance to press the 'capture' button (and thus before the photo would have been saved to disk). This will cause a file-not-found error (because the file has not been created yet), or will cause previously captured photo to be displayed.